# Quantity based policy setup

## Setting quantity path

### Define values

Here we are interested in quantity-based policy. Note that each model has quantity-based policy examples restricting maximum carbon emission ("cap") in either global or regional consideration. Yet all model has default setting to trade between regions. This makes carbon emission constraint becomes practically global. Still, different models has different cap profiles and possibly what kind of carbon (fossil-fuel vs. total) is accounted. To make a fair comparison, we need to set a consistent quantity cap path and make same carbon accounting.

In [1]:
import pandas as pd
import pandas.io.excel
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pytz
import os
import patsy #for spline regression
import scipy #for non-negative least square 
import scipy as sp 
from scipy import stats
from scipy.optimize import nnls
from numpy.linalg import inv #for matrix and statistics
import scipy as sp
import math
import statsmodels.api as sm
%matplotlib inline

First, we set consistent global carbon cap in unit million metric ton of CO2 (MMtCO2). We may start by considering carbon cap path provided by EPPA. In EPPA, there is an initialization by multiplying quantity called $\texttt{co2_ref("2000",r)}$ in million tons of C with reduction factor called $\texttt{reduc(t)}$. Initially this generates cap for fossil fuel carbon emission for all regions and time, but we may use it as a cap all carbon emission, for example, in MERGE. Here we summarize such information as a global cap path in MMtCO2 starting from 2020 and the distribution factor for each region as follow:

In [2]:
EPPAbased_cap_list = [31068.18659,32639.29401,33343.5814,35348.9658,35573.2948,35484.98783,35211.65708,35239.58841,34702.2682,33176.07236,\
                 30894.95337,28637.3722,26150.97844,24596.35871,22608.3863,21203.61001,19952.4271]
EPPAregionlist = ['USA','CAN','MEX','JPN','ANZ',\
                  'EUR','EET','FSU','ASI','CHN',\
                  'IND','IDZ','AFR','MES','LAM','ROW']
EPPAregionfactor = [0.248842792,0.024377986,0.016295038,0.049593052,0.015196566,\
                    0.157608941,0.033799017,0.084971317,0.053151416,0.127656132,\
                    0.042218042,0.010402809,0.038865058,0.038549208,0.036780139,0.021694043] 

When we apply this to MERGE which defines carbon cap by region, we need to map this region distribution factor for GCAM as follow:

In [3]:
MERGEregionlist = ['usa','eur','japan','canz','russia','china','india','xas','row']         
# 'usa' = 'USA', 'eur' = 'EUR'+'EET', 'japan'='JPN',
#'canz' = 'CAN'+'ANZ', 'russia'='FSU', 'china'='CHN',
#'india'='IND','xas'='ASI'+'IDZ'+'MES','row'='MEX'+'LAM'+'ROW'+'AFR'
MERGEregionfactor = [0.248842792, 0.157608941+0.033799017, 0.049593052,\
                    0.024377986+0.015196566, 0.084971317, 0.127656132,\
                    0.042218042, 0.053151416+0.010402809+0.038549208, 0.016295038+0.038865058+0.036780139+0.021694043] 

Here is the cap path provided by EPPA.

In [4]:
startyear = 2020
perturbation = 1.00
EPPAbased_cap = pd.DataFrame(columns=['year','MMtCO2']) 
EPPAbased_cap['year'] =  [startyear+5*i for i in range(0,17)]
EPPAbased_cap['MMtCO2'] = EPPAbased_cap_list

In [5]:
EPPAbased_cap[EPPAbased_cap.year==2050]

,year,MMtCO2
6,2050,35211.65708


One may construct another cap path, say, step cap. We need a value of step cap and the year where step occurs. We may try step of 30000, 45000 and 60000 MMtCO2 with step at year 2050. Here is an example.

In [6]:
c_ini = 10000.0 #60000.0, 50000.0, 40000.0, 30000.0
r = 1
startyear = 2020
stepyear = 2050
perturbation = 1.00
step_cap = pd.DataFrame(columns=['year','MMtCO2']) 
step_cap['year'] =  [startyear+5*i for i in range(0,17)]
step_cap['MMtCO2'][step_cap.year<stepyear] = float('Inf')
step_cap['MMtCO2'][step_cap.year>=stepyear] = c_ini*perturbation*np.power(r,(i*5))

In [7]:
step_cap

,year,MMtCO2
0,2020,inf
1,2025,inf
2,2030,inf
3,2035,inf
4,2040,inf
5,2045,inf
6,2050,10000
7,2055,10000
8,2060,10000
9,2065,10000


For MERGE, set cap, namely $\texttt{carlim}$ in 'm7.gms' has same unit as other carbon emission terms in MERGE, which is billion tons of C. So we need conversion $\text{set cap} = \text{MMtCO2 emission}/44*12/1000$. 

For EPPA, set cap, namely $\texttt{cquota}$ in .cas file is assigned based on  $\texttt{co2_ref("2000",r)}$ in million tons of C. We will replace this parameter with our number in the same unit. Hence, we need conversion: $\text{set cap} = \text{MMtCO2 emission}/44*12$.

For GCAM, set cap is million tons of C in 'Main_User_Workspace/input/policy/carbon_cap_450.xml' file and it needs conversion: $\text{set cap} = \text{MMtCO2 emission}/44*12$.

We can define function called 'cap_path_conversion' to generate conversion for any cap path:

In [8]:
def cap_path_conversion(captable):
    captable['MERGE set cap'] = captable['MMtCO2']/44*12/1000
    captable['EPPA set cap'] = captable['MMtCO2']/44*12
    captable['GCAM set cap'] = captable['MMtCO2']/44*12
    return captable;

In [9]:
#cap_path_conversion(EPPAbased_cap)

In [10]:
cap_path_conversion(step_cap)

,year,MMtCO2,MERGE set cap,EPPA set cap,GCAM set cap
0,2020,inf,inf,inf,inf
1,2025,inf,inf,inf,inf
2,2030,inf,inf,inf,inf
3,2035,inf,inf,inf,inf
4,2040,inf,inf,inf,inf
5,2045,inf,inf,inf,inf
6,2050,10000,2.727273,2727.273,2727.273
7,2055,10000,2.727273,2727.273,2727.273
8,2060,10000,2.727273,2727.273,2727.273
9,2065,10000,2.727273,2727.273,2727.273


In [34]:
2727.273*4*1

10909.092

### How to modify cap paths in models with perturbation

$\underline{\textbf{WARNING}}$ Some of modifications here is a dirty way. Don't forget to copy original file and name it, say, '[original name]_original' before doing any change.

$\underline{\text{EPPA}}$:
1. In quantity-based policy .cas file, define extra parameter called perturb in order to vary cap path and measure elasticity Here we use perturb from 0.85 to 1.15 with step size 0.05. Next define another parameter regionfactor in order to distribute the cap to all regions. Note that the value of each region can be anything between 0 and 1 as long as the sum equals 1. Then assign the cap cquota(r,t)$(ord(t) ge 12)= regionfactor(r) * perturb * [cap-value]; after original cquota assignment. Save .cas file with name test_policy_quantity_[step cap name]_[perturb].cas

2. Make sure to use original eppaloop2.gms.

3. To run multiple cases with different values of perturbation, create a batch file, for example, batch_quantity-based.dat. Each line should read: call rrun [.cas file name without .cas]



$\underline{\text{GCAM}}$:

1. Follow an instruction of running reference case.

2. Start from quantity-based policy .xml file in $\texttt{/input/policy}$ folder, for example, $\texttt{carbon_cap_450.xml}$. Add lines of fixed cap for individual year: $\texttt{<constraint year="2050">[cap-value]</constraint>}$. This cap value should include perturbation. Don't forget to save this new input policy file with a distinct name.

3. Start from original $\texttt{configuration.xml}$ in exe folder. Modify it as follow:

    -Add policy in ScenarioComponents object as: $\texttt{<Value name = "policy">../input/policy/[input-policy-name].xml</Value>}$ 
    
    -Change scenarioName in Strings object as: $\quad\texttt{<Value name="scenarioName">[input-policy-name]</Value>}$. This will help us find scenarios easier in Java interface.

    -Change CalibrationActive in Bools object to be 0 in order to prevent calibration change.

4. To run multiple cases with different values of perturbation, create a batch file, for example, $\texttt{batch_quantity-based.dat}$. Each scenario should run $\texttt{Objects-Main}$.



$\underline{\text{MERGE}}$: 
1. Add step cap path. In m7.gms, define two parameters carlim_step for global cap and regionfactor in order to distribute the cap to all regions. Note that the value of each region can be anything between 0 and 1 as long as the sum equals one. 

2. Add perturbation as follow:

    -In m7.gms define extra parameter called perturb in order to vary price path and measure elasticity. perturb ranges from 0.85 to 1.15 with step size 0.05. 
    
    -In studies.xls, go to Lexicon sheet and add new lexicon perturb with relevant information, then go to aero sheet and add extra column for perturb lexicon.
    
    - In m7.gms add extra line after assigning carbon cap to perturb the cap as follow: $\texttt{\$if not set ametax and set perturb carlim(tp,rg) = carlim_step(tp)* regionfactor(rg) * perturb("%perturb%");}$

3. To run multiple cases with different values of perturbation, add multiple lines in studies.xls modifying from the baseline case.



## Carbon accounting issue: fossil fuel only (FOS), fossil and industry (FI), and total (TOT)

See detail in Notebook 02_00_PriceBasedPolicyStep